In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import movingpandas as mpd
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi

In [ ]:
# Load receiver array positions
receivers_gdf = pd.read_csv('../data/VPS-Station-Locations.csv')
receivers_gdf['geometry'] = gpd.points_from_xy(receivers_gdf.Lng, receivers_gdf.Lat)
receivers_gdf = gpd.GeoDataFrame(receivers_gdf)
receivers_gdf = receivers_gdf.set_crs('EPSG:4326')
receivers_gdf.head()

In [ ]:
# Set the filename
# filename = '../data/SharkArray-2020-05-21-thru-05-28.csv'
filename = '../data/SharkArray-01-animals.csv'

# Load shark positions data into a GeoDataFrame
shark_gdf = pd.read_csv(filename)
shark_gdf['t'] = pd.to_datetime(shark_gdf['DATETIME'])
shark_gdf['geometry'] = gpd.points_from_xy(shark_gdf.LON, shark_gdf.LAT)
shark_gdf = gpd.GeoDataFrame(shark_gdf)
shark_gdf = shark_gdf.set_crs('EPSG:4326')
shark_gdf = shark_gdf.set_index('t').tz_localize(None)
shark_gdf.head()

In [ ]:
print('The dataset contains', shark_gdf.shape[0], 'rows and', shark_gdf.shape[1], 'columns.')
print('The column names are:', list(shark_gdf.columns.values))
print('The unique transmitter names are:', shark_gdf['TRANSMITTER'].unique())

In [ ]:
# Create separate trajectories for each shark based on their transmitter ID
traj_collection = mpd.TrajectoryCollection(shark_gdf, 'TRANSMITTER')
print(traj_collection)

In [ ]:
# Get number of points in each trajectory
data = []
for traj in traj_collection:
    data.append([traj.id.split('_')[0], traj.df.shape[0]])
num_points = pd.DataFrame(data, columns=['TRANSMITTER', 'NUM_POSITIONS'])
print(num_points)

In [ ]:
# Add a timedelta column which is the time between the previous position and the current position
for traj in traj_collection.trajectories:
    n = traj.df.shape[0]
    timedeltas = [timedelta()] + [traj.df.index[i] - traj.df.index[i - 1] for i in range(1, n)]
    traj.df['TIMEDELTA'] = timedeltas

In [ ]:
# Plot timemap of positions
cmap = plt.get_cmap('jet')
N = len(traj_collection.trajectories)
fig, axs = plt.subplots(N + 1, 1, figsize=(5, 5 * (N + 1)))
for i, traj in enumerate(traj_collection.trajectories):
    color = cmap(float(i) / N)
    seconds = np.array([traj.df['TIMEDELTA'][i].total_seconds() for i in range(1, traj.df.shape[0])])
    xcoords = seconds[:-1] / 60
    ycoords = seconds[1:] / 60
    axs[0].plot(xcoords, ycoords, marker='.', ls='', markerfacecolor=color, markeredgecolor=color, label=traj.id.split('_')[0])
    axs[i + 1].plot(xcoords, ycoords, marker='.', ls='', markerfacecolor=color, markeredgecolor=color, label=traj.id.split('_')[0])
    axs[i + 1].set_title('Num points: ' + str(len(seconds)))
ticks = [1, 3, 10, 60, 600, 1440, 14400]
tick_labels = ['1 min', '3 min', '10 min', '1 hr', '10 hr', '1 d', '10 d']
for ax in axs:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.legend(bbox_to_anchor=(1.04, 1))
    ax.set_xlabel('Time since last position')
    ax.set_ylabel('Time before next position')
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(tick_labels)
    ax.set_yticklabels(tick_labels)
axs[0].set_title('Shark Array 01 Time Map')
plt.savefig('../animations/scatter_timemaps_full_dataset', bbox_inches="tight")
plt.show()

In [ ]:
OUTPUT_SCALE = 'log'

# Extract a single trajectory
seconds = []
xcoords = []
ycoords = []
transform_data = lambda x: np.log10(x / 60)
for i, traj in enumerate(traj_collection.trajectories):
    seconds += [traj.df['TIMEDELTA'][i].total_seconds() for i in range(1, traj.df.shape[0])]
    xcoords += seconds[:-1]
    ycoords += seconds[1:]
seconds = np.array(seconds)
xcoords = transform_data(np.array(xcoords))
ycoords = transform_data(np.array(ycoords))

# Set up heatmap

# smooth
bins = 256
width = 8

# granular
# bins = 50
# width = 0

H = np.zeros((bins, bins))
max_diff = transform_data(np.max(seconds))
x_heat = (bins - 1) * xcoords / max_diff
y_heat = (bins - 1) * ycoords / max_diff
for i in range(len(xcoords)):
    H[int(x_heat[i]), int(y_heat[i])] += 1
H = ndi.gaussian_filter(H, width)

if OUTPUT_SCALE == 'log':
    # Log scale output
    transform_output = lambda x: np.log10(x + 1)
    transform_output_inv = lambda x: 10 ** x - 1
else:
    # Linear output
    transform_output = lambda x: x
    transform_output_inv = lambda x: x

H = transform_output(H.T)

# Plot
cmap = plt.get_cmap('nipy_spectral_r')
fig, ax = plt.subplots(figsize=(5, 5))
plt.imshow(H, origin='lower', extent=(0, max_diff, 0, max_diff), cmap=cmap)
ax.set_xlim(0, max_diff)
ax.set_ylim(0, max_diff)
ax.set_xlabel('Time since last position')
ax.set_ylabel('Time before next position')
ax.set_title('Shark Array 01 Time Map')
ticks = transform_data(np.array([1, 2, 3, 10, 60, 600, 1440, 14400]) * 60)
tick_labels = ['1 min', '', '3 min', '10 min', '1 hr', '10 hr', '1 d', '10 d']
plt.xticks(ticks, tick_labels)
plt.yticks(ticks, tick_labels)
cbar = plt.colorbar()
cbar.set_label('Num positions')

# cbar_ticks = transform_output(np.linspace(0, transform_output_inv(np.max(H)), 10))
# cbar_tick_labels = ['%.2f' % x for x in transform_output_inv(cbar_ticks)]

cbar_ticks = np.linspace(0, np.max(H), 10)
cbar_tick_labels = ['%.2f' % x for x in transform_output_inv(cbar_ticks)]

cbar.set_ticks(cbar_ticks)
cbar.set_ticklabels(cbar_tick_labels)

plt.savefig('../animations/heatmap_time_map_full_dataset_smooth', bbox_inches="tight")
plt.show()

In [ ]:
# Create histogram of position data by day
start_time = shark_gdf.index.min()
end_time = shark_gdf.index.max()
delta = end_time - start_time
bins = int(delta.total_seconds() // (3600 * 24))
data = np.array([(shark_gdf.index[i] - start_time).total_seconds() for i in range(shark_gdf.shape[0])]) / (3600 * 24)
plt.hist(data, bins=bins)
# plt.xticks(np.linspace(0, 7, 8) * 24, ['5-21', '5-22', '5-23', '5-24', '5-25', '5-26', '5-27', '5-28'])
plt.gca().set_title('Position data histogram')
plt.show()

In [ ]:
start_time = shark_gdf.index.min()
end_time = shark_gdf.index.max()
delta = end_time - start_time
num_bins = int(delta.total_seconds() // (3600 * 24))
total_data = []
cmap = plt.get_cmap('jet')
N = len(traj_collection.trajectories)
fig, axs = plt.subplots(N + 1, 1, figsize=(5, 5 * (N + 1)))
ax.hist(data, bins=num_bins)
total_data = []
colors = []
labels = []
for (i, traj), ax in zip(enumerate(traj_collection.trajectories), axs[1:]):
    color = cmap(float(i) / N)
    data = np.array([(traj.df.index[i] - start_time).total_seconds() for i in range(traj.size())]) / (3600 * 24)
    label = traj.id.split('_')[0]
    total_data.append(data)
    colors.append(color)
    labels.append(label)
    ax.hist(data, bins=num_bins, color=color, label=label)
    ax.legend(bbox_to_anchor=(1.04, 1))
    ax.set_title('Num positions: ' + str(traj.size()))
axs[0].hist(total_data, bins=num_bins, color=colors, label=labels, stacked=True, histtype='barstacked')
axs[0].legend(bbox_to_anchor=(1.04, 1))

# Set the same scale for each plot
for ax in axs[1:]:
    ax.set_ylim(axs[0].get_ylim())

# plt.xticks(np.linspace(0, 7, 8) * 24, ['5-21', '5-22', '5-23', '5-24', '5-25', '5-26', '5-27', '5-28'])
axs[0].set_title('Position data histograms')
plt.savefig('../animations/position_data_histograms_full_dataset', bbox_inches="tight")
plt.show()

In [ ]:
plt.hist(seconds / 60, bins = list(range(0, 60)))
plt.gca().set_title('Time difference histogram')
plt.gca().set_xlabel('minutes')
plt.show()

In [ ]:
# Add velocities and headings to each trajectory
for traj in traj_collection.trajectories:
    traj.add_speed()
    traj.add_direction()

In [ ]:
# Compute turning angles
def bound_angle_diff(theta_diff):
    return ((theta_diff - 180) % 360) - 180

for traj in traj_collection.trajectories:
    n = traj.df.shape[0]
    turning_angles = [traj.df['direction'][0]] + [bound_angle_diff(traj.df['direction'][i + 1] - traj.df['direction'][i]) for i in range(1, n - 1)] + [0]
    traj.df['turning_angle'] = turning_angles

In [ ]:
traj_collection.trajectories[0].df.head()

In [ ]:
# Verify turning angle calculation by showing segment of trajectory
i = 1
traj_collection.trajectories[0].get_linestring_between(traj_collection.trajectories[0].df.index[i - 1], traj_collection.trajectories[0].df.index[i + 1])

In [ ]:
# Split by observation gap
obs_gap_trajs_dict = {}
for traj in traj_collection:
    obs_gap_trajs_dict[traj.id.split('_')[0]] = mpd.ObservationGapSplitter(traj).split(gap=timedelta(minutes=3))

In [ ]:
# Print lengths of trajectories
obs_gap_lengths_dict = {}
data = []
for name, obs_gap_trajs in obs_gap_trajs_dict.items():
    lengths = np.array([obs_gap_trajs.trajectories[i].size() for i in range(len(obs_gap_trajs.trajectories))])
    obs_gap_lengths_dict[name] = lengths
    print(name)
    print(lengths)
    print('max:', np.max(lengths), 'min:', np.min(lengths), 'avg:', '%.2f' % np.average(lengths), 'sum:', np.sum(lengths), 'len:', len(lengths))
    idx = np.argmax(lengths)
    max_traj = obs_gap_trajs_dict[name].trajectories[idx]
    data.append([name, np.max(lengths), max_traj.get_start_time(), max_traj.get_end_time(), np.average(lengths), np.sum(lengths), len(lengths)])
obs_gap_lengths_df = pd.DataFrame(data, columns=['TRANSMITTER', 'MAX_LEN', 'MAX_START_TIME', 'MAX_END_TIME', 'AVG', 'SUM', 'NUM'])

In [ ]:
print(obs_gap_lengths_df)

In [ ]:
np.argmax(obs_gap_lengths_dict['2020-21'])

In [ ]:
obs_gap_trajs_dict['2020-21'].trajectories[509].df['TIMEDELTA']